UC3MAL201 Machine Learning Session 11 Tutorial 11 Leon Eriksen Helgeland -
First written 09.09.2020

# Neural Network - with Keras

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import KFold
import tensorflow.keras.optimizers
import os

In [3]:
import tensorflow
print(tensorflow.__version__)

2.4.0-dev20200929


In [4]:
dataset_train_file = 'mnist_train.csv'
dataset_test_file = 'mnist_test.csv'

dataset_directory = r'C:\Users\leon.helgeland\OneDrive\S05-MAL-2020\S05MAL - Session 11 - Neural Network with Keras - Live Demo\Dataset for the tutorial-20200908'#'~/GitHub/BigData/dataset11'

In [5]:
# import train and test dataset
train = pd.read_csv(f'{dataset_directory}/{dataset_train_file}',header=None)
test = pd.read_csv(f'{dataset_directory}/{dataset_test_file}',header=None)

## Dataset preparation
Rescale the data to values between 0.01 and 1, by multiplying each pixel value in the dataset by (0.99/255) and adding 0.01 to the result to get a new pixel value.

In [6]:
# Import dataset
X_train = train.iloc[:,1:].apply(lambda a: a*(0.99/255)+0.01)
y_train = train.iloc[:,:1]
X_test = test.iloc[:,1:].apply(lambda a: a*(0.99/255)+0.01)
y_test = test.iloc[:,:1]

In [7]:
# The class labels are integer values, encoding them into one-hot representation https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories="auto", sparse=False, dtype=np.int16) # handle_unknown='ignore'
y_train = enc.fit_transform(y_train.to_numpy().reshape(-1, 1))
y_test = enc.fit_transform(y_test.to_numpy().reshape(-1, 1))

In [8]:
def model_fit(X_train, y_train, config):
    n_input = config[0]
    n_output = config[1]
    n_nodes = config[2]
    epoch = config[3]
    batch = config[4]
    # Define and Compile
    model = Sequential()
    model.add(Dense(n_nodes, input_dim=n_input, activation='relu'))
    model.add(Dense(n_output, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X_train, y_train, epochs=epoch, batch_size=batch)
    return model

# predict with the fit model
def model_predict(model, testdata):
    Prediction = model.predict(testdata, verbose=0)
    return Prediction

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [9]:
inputs = 784 # Input features
outputs = 10 # Output features
nodes = [5,10,15,20,35,50] # Test amount of nodes in layer

configs = []
for n in nodes:
    config_ = [inputs, outputs, n, 75, 250]
    configs.append(config_)

In [11]:
# Kfolding
config_lossess = []
accuracy_per_config = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)

counter = 0
for config in configs:
    counter+=1
    fold_count = 0

    fold_losses = []
    fold_accuracy = []
    
    for fold_train, fold_test in kf.split(X_train, y_train):
        fold_count+=1
        nn_model = model_fit(X_train[fold_train], y_train[fold_train], config)
        scores = neural_network_model.evaluate(X_train[fold_test], y_train[fold_test], verbose=0)
        print(f"Score for config {counter} fold {fold_count}: {neural_network_model.metrics_names[0]}={scores[0]}; {neural_network_model.metrics_names[1]}={scores[1]*100}%")
        
        fold_losses.append(scores[0])
        fold_accuracy.append(scores[1]*100)

    config_lossess.append(fold_losses)
    accuracy_per_config.append(fold_accuracy)

KeyError: &#39;[0, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 801, 804, 805, 806, 808, 809, 810, 813, 814, 815, 816, 818, 819, 820, 822, 823, 824, 825, 826, 827, 830, 831, 832, 834, 835, 837, 838, 839, 840, 841, 843, 844, 845, 846, 847, 851, 852, 854, 855, 856, 858, 859, 860, 862, 863, 864, 865, 867, 868, 869, 870, 871, 872, 873, 875, 876, 877, 878, 880, 881, 882, 883, 885, 886, 887, 888, 889, 891, 893, 894, 895, 897, 899, 900, 901, 902, 903, 907, 908, 909, 910, 911, 912, 915, 916, 917, 918, 919, 921, 922, 923, 924, 925, 928, 929, 931, 932, 933, 934, 935, 937, 938, 940, 941, 944, 945, 946, 947, 948, 950, 951, 952, 954, 956, 958, 959, 960, 961, 963, 964, 966, 968, 969, 970, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 986, 987, 988, 989, 990, 991, 992, 993, 995, 996, 997, 999, 1001, 1002, 1004, 1006, 1007, 1008, 1009, 1010, 1011, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1031,

In [ ]:
print("Score per config: ")
for j in range(0, len(accuracy_per_config)):
    print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
    print(f'Score for config {j+1}. Nodes:{configs[j][2]}')
    print('Average scores for all folds:')
    print(f'> Loss: {np.mean(config_lossess[j])}')
    print(f'> Accuracy: {np.mean(accuracy_per_config[j])} (+- {np.std(accuracy_per_config[j])})')
    print('')

In [10]:
configy = [784, 10, 20, 10, 1]
neural_network = model_fit(X_train, y_train, configy)

Epoch 1/10
60000/60000 [==============================] - 28s 466us/step - loss: 0.4318 - accuracy: 0.8710
Epoch 2/10
60000/60000 [==============================] - 28s 465us/step - loss: 0.2129 - accuracy: 0.9382
Epoch 3/10
60000/60000 [==============================] - 28s 472us/step - loss: 0.1840 - accuracy: 0.9469
Epoch 4/10
60000/60000 [==============================] - 28s 465us/step - loss: 0.1645 - accuracy: 0.9529
Epoch 5/10
60000/60000 [==============================] - 28s 471us/step - loss: 0.1546 - accuracy: 0.9549
Epoch 6/10
60000/60000 [==============================] - 28s 465us/step - loss: 0.1484 - accuracy: 0.9586
Epoch 7/10
60000/60000 [==============================] - 29s 475us/step - loss: 0.1437 - accuracy: 0.9609
Epoch 8/10
60000/60000 [==============================] - 28s 464us/step - loss: 0.1364 - accuracy: 0.9615
Epoch 9/10
60000/60000 [==============================] - 28s 472us/step - loss: 0.1342 - accuracy: 0.9618
Epoch 10/10
60000/60000 [============

In [11]:
# Evaluate the model
prediction = model_predict(neural_network, X_test)

PredClass = list()
ActualClass = list()
for row in range(len(y_test)):
    index_maxscore1 = max(range(len(y_test[row])), key=y_test[row].__getitem__)
    ActualClass.append(index_maxscore1)
    index_maxscore2 = max(range(len(prediction[row])), key=prediction[row].__getitem__)
    PredClass.append(index_maxscore2)
    print('Expected=%d, Got=%d' % (index_maxscore1, index_maxscore2))


accuracy = accuracy_metric(PredClass, ActualClass)
accuracy

6
Expected=2, Got=2
Expected=6, Got=6
Expected=7, Got=7
Expected=1, Got=1
Expected=9, Got=9
Expected=8, Got=8
Expected=0, Got=0
Expected=6, Got=6
Expected=9, Got=9
Expected=4, Got=4
Expected=9, Got=9
Expected=9, Got=9
Expected=6, Got=6
Expected=2, Got=2
Expected=3, Got=3
Expected=7, Got=7
Expected=1, Got=1
Expected=9, Got=9
Expected=2, Got=2
Expected=2, Got=2
Expected=5, Got=5
Expected=3, Got=3
Expected=7, Got=7
Expected=8, Got=8
Expected=0, Got=0
Expected=1, Got=1
Expected=2, Got=2
Expected=3, Got=3
Expected=4, Got=4
Expected=7, Got=7
Expected=8, Got=8
Expected=9, Got=9
Expected=0, Got=0
Expected=1, Got=1
Expected=2, Got=2
Expected=3, Got=3
Expected=4, Got=4
Expected=7, Got=7
Expected=8, Got=8
Expected=9, Got=9
Expected=0, Got=0
Expected=1, Got=1
Expected=7, Got=7
Expected=8, Got=8
Expected=9, Got=9
Expected=8, Got=8
Expected=9, Got=9
Expected=2, Got=2
Expected=6, Got=6
Expected=1, Got=1
Expected=3, Got=3
Expected=5, Got=5
Expected=4, Got=4
Expected=8, Got=8
Expected=2, Got=2
Expected

95.14

In [1]:
    import tensorflow as tf
    if tf.test.gpu_device_name():
        print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
    else:
        print("Please install GPU version of TF")

Please install GPU version of TF
